In [1]:
import pandas as pd
import datetime

In [2]:
df = pd.read_parquet('../data/forecasting_format/cases_per_day_forecasting_format.parquet')

### Adding accumulated cases as cases_tot

In [3]:
df_covid_cases = pd.read_parquet('../data/pre_processed/data_covid_cases.parquet')

In [4]:
all_countries = list(df_covid_cases.columns)

In [5]:
df_cases_tot = pd.DataFrame(columns=['Cases','Country'])

In [6]:
df_cases_tot

,Cases,Country


In [7]:
for country in all_countries:
    temp_segment = pd.DataFrame(df_covid_cases[country])
    temp_segment.rename(columns={country:"Cases"}, inplace=True)
    temp_segment['Country']=country
    temp_segment['Date'] = temp_segment.index
    df_cases_tot = pd.concat([df_cases_tot, temp_segment], ignore_index = True)

In [8]:
df_cases_tot

,Cases,Country,Date
0,3.0,Algeria,2020-03-01
1,3.0,Algeria,2020-03-02
2,3.0,Algeria,2020-03-03
3,5.0,Algeria,2020-03-04
4,12.0,Algeria,2020-03-05
...,...,...,...
16276,7508.0,Zimbabwe,2020-09-13
16277,7526.0,Zimbabwe,2020-09-14
16278,7531.0,Zimbabwe,2020-09-15
16279,7576.0,Zimbabwe,2020-09-16


In [9]:
df['cases_tot'] = df_cases_tot['Cases']

### Changing index to be Date column

In [10]:
df.set_index('Date', inplace=True)

In [11]:
df.head(5)

,Cases,Country,cases_tot
Date,,,
2020-03-01,3.0,Algeria,3.0
2020-03-02,0.0,Algeria,3.0
2020-03-03,0.0,Algeria,3.0
2020-03-04,2.0,Algeria,5.0
2020-03-05,7.0,Algeria,12.0


### Add Month, Day and Weekend column

Value 1-12 depending of month 

In [12]:
df['Month'] = df.index.month

Value 0-6 depending on day in week, where Monday is 0 and Sunday 6

In [13]:
df['Day'] = df.index.weekday

Value 0/1 depending on weekend or weekday. 0: weekday, 1: weekend.

In [14]:
df['Weekend'] = ((pd.DatetimeIndex(df.index).dayofweek) // 5 == 1).astype(float)

### Add previous case

Add previous nr of cases to nex row. Sett first day as 0 

In [15]:
df['prev_case'] = df['Cases'].shift(periods=1)

In [16]:
startDate = datetime.datetime(2020, 3, 1)

In [17]:
df.prev_case[df.index==startDate] = 0

<ipython-input-17-695e3c0af246>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.prev_case[df.index==startDate] = 0


### Result dataframe with the new features:

In [18]:
df.head(5)

,Cases,Country,cases_tot,Month,Day,Weekend,prev_case
Date,,,,,,,
2020-03-01,3.0,Algeria,3.0,3,6,1.0,0.0
2020-03-02,0.0,Algeria,3.0,3,0,0.0,3.0
2020-03-03,0.0,Algeria,3.0,3,1,0.0,0.0
2020-03-04,2.0,Algeria,5.0,3,2,0.0,0.0
2020-03-05,7.0,Algeria,12.0,3,3,0.0,2.0


In [19]:
#df.to_parquet('../data/forecasting_format/dataframe_forecasting_with_features.parquet')